In [4]:
import pandas as pd

df_elec_cars = pd.read_csv("../data/data.csv")

In [8]:
df_elec_cars.head(7)

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,E.V_Type,CAFV,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,JTMAB3FV3P,Kitsap,Seabeck,WA,98380.0,2023,TOYOTA,RAV4 PRIME,PHEV,known,42.0,0.0,35.0,240684006,POINT (-122.8728334 47.5798304),PUGET SOUND ENERGY INC,5.303509e+10
1,1N4AZ1CP6J,Kitsap,Bremerton,WA,98312.0,2018,NISSAN,LEAF,BEV,known,151.0,0.0,35.0,474183811,POINT (-122.6961203 47.5759584),PUGET SOUND ENERGY INC,5.303508e+10
2,5YJ3E1EA4L,King,Seattle,WA,98101.0,2020,TESLA,MODEL 3,BEV,known,266.0,0.0,43.0,113120017,POINT (-122.3340795 47.6099315),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
3,1N4AZ0CP8E,King,Seattle,WA,98125.0,2014,NISSAN,LEAF,BEV,known,84.0,0.0,46.0,108188713,POINT (-122.304356 47.715668),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303300e+10
4,1G1FX6S00H,Thurston,Yelm,WA,98597.0,2017,CHEVROLET,BOLT EV,BEV,known,238.0,0.0,20.0,176448940,POINT (-122.5715761 46.9095798),PUGET SOUND ENERGY INC,5.306701e+10
5,5YJYGDEE5L,Snohomish,Lynnwood,WA,98036.0,2020,TESLA,MODEL Y,BEV,known,291.0,0.0,21.0,124511187,POINT (-122.287143 47.812199),PUGET SOUND ENERGY INC,5.306105e+10
6,KM8S6DA23N,Kitsap,Poulsbo,WA,98370.0,2022,HYUNDAI,SANTA FE,PHEV,known,31.0,0.0,23.0,212217764,POINT (-122.6368884 47.7469547),PUGET SOUND ENERGY INC,5.303509e+10


In [12]:
# List comprehension para criar uma lista apenas com as colunas que possuem valores nulos

nan_columns = [columns for columns in df_elec_cars.columns if df_elec_cars[columns].isna().sum() > 0]

In [13]:
nan_columns

['County',
 'City',
 'Postal Code',
 'Model',
 'Electric Range',
 'Base MSRP',
 'Legislative District',
 'Vehicle Location',
 'Electric Utility',
 '2020 Census Tract']